In [1]:
import numpy as np
import cv2 as cv
import matplotlib
from matplotlib import pyplot as plt

In [2]:
img_name = 'img/lena_std.tif'
img = cv.imread(img_name, cv.IMREAD_GRAYSCALE)

hist_2d, xbins, ybins = np.histogram2d(img.ravel(), cv.blur(img, (3, 3)).ravel(), bins=256, range=((0, 256), (0, 256)), density=True)

In [7]:
def max_entropy_2d(hist_2d):
    """
    Implements AHMED S. ABLJTALEB* (2d Maximum Entropy) thresholding method
    AHMED S. ABLJTALEB* (1989) "Automatic Thresholding of Gray-Level Pictures Using Two-Dimensional Entropy"
    Params:
        hist_2d [np.array]: 归一化后的二维直方图，i：像素灰度值，j：像素3*3邻域平均灰度值。
    Return:
        threshold [int]: threshold calculated by 2维最大熵算法
    """
    pdf_normal = hist_2d.cumsum().reshape(hist_2d.shape)

    nonzero_indices = np.nonzero(hist_2d)
    i_start, i_end = nonzero_indices[0][0], nonzero_indices[0][-1]
    j_start, j_end = nonzero_indices[1][0], nonzero_indices[1][-1]

    total_range = hist_2d[hist_2d != 0]
    H_mm = -np.sum(total_range * np.log(total_range))
    max_ent, threshold = 0, 0
    for i in range(i_start, i_end):
        for j in range(j_start, j_end):
            P_st = pdf_normal[i][j]
            st_range = hist_2d[:i+1,:j+1]
            st_range = st_range[st_range != 0]
            H_st = -np.sum(st_range * np.log(st_range))
            total_ent = np.log(P_st * (1 - P_st)) + H_st/P_st + (H_mm - H_st)/(1 - P_st)
            # find max
            if total_ent > max_ent:
                max_ent, threshold = total_ent, i

    return threshold

In [8]:
hist_2d.shape

(256, 256)